In [3]:
import pandas as pd

#File too large for Github - Create Relative Path
df = pd.read_csv('C:\\Users\\Ben\\OneDrive\\Documents\\MarylandData\\Maryland_Statewide_Vehicle_Crashes.csv')
df = df[['YEAR', 'COUNTY_DESC', 'REPORT_TYPE', 'ACC_DATE', 'ACC_TIME', 'LATITUDE', 'LONGITUDE']]
df['COUNTY_DESC'] = df['COUNTY_DESC'].str.replace("'", "")

# Convert ACC_DATE to datetime format
df['ACC_DATE'] = pd.to_datetime(df['ACC_DATE'], format='%Y%m%d')
# Combine ACC_DATE and ACC_TIME into a single datetime column
df['ACC_DATETIME'] = pd.to_datetime(df['ACC_DATE'].astype(str) + ' ' + df['ACC_TIME'])

# Extract hour and month from ACC_DATETIME
df['HOUR'] = df['ACC_DATETIME'].dt.hour
df['MONTH'] = df['ACC_DATETIME'].dt.month
df['DAY'] = df['ACC_DATETIME'].dt.dayofweek
df = df.drop(columns=['ACC_DATETIME','ACC_TIME'])
df_small = df[df['YEAR'].isin([2015,2016,2017,2018,2019,2020]) == False]

C:\Users\Ben\AppData\Local\Temp\ipykernel_29520\1448207667.py:4: DtypeWarning: Columns (33,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:\\Users\\Ben\\OneDrive\\Documents\\MarylandData\\Maryland_Statewide_Vehicle_Crashes.csv')


In [4]:
df_small.to_csv('DF_Output_Processed.csv', index=False)

In [5]:
# Pre-compute the data for the plots
hourly_counts = df.groupby(['HOUR', 'COUNTY_DESC', 'REPORT_TYPE']).size().reset_index()
hourly_counts.columns = ['HOUR', 'COUNTY_DESC', 'REPORT_TYPE', 'COUNT']
county_counts = df.groupby(['COUNTY_DESC', 'REPORT_TYPE']).size().unstack(fill_value=0).reset_index()

#For Long Term Trends
daily_counts = df.groupby(['ACC_DATE', 'COUNTY_DESC']).size().reset_index()
daily_counts['ACC_DATE'] = pd.to_datetime(daily_counts['ACC_DATE'])
daily_counts.columns = ['ACC_DATE', 'COUNTY_DESC', 'COUNT']
daily_counts.reset_index(inplace=True)
daily_counts['day_of_year'] = daily_counts['ACC_DATE'].dt.dayofyear
pre_covid = daily_counts[daily_counts['ACC_DATE'] < '2020-03-1']
post_covid = daily_counts[daily_counts['ACC_DATE'] >= '2020-03-1']
pre_covid_avg = pre_covid.groupby('day_of_year')['COUNT'].mean()
post_covid = post_covid.merge(pre_covid_avg, on='day_of_year', how='left', suffixes=('_post', '_pre'))
post_covid['percentage_of_pre_covid'] = (post_covid['COUNT_post'] / post_covid['COUNT_pre']) * 100
post_covid['Day_of_Year'] = post_covid['ACC_DATE'].dt.dayofyear
post_covid['Year'] = post_covid['ACC_DATE'].dt.year
post_covid['percentage_of_pre_covid_smooth'] = post_covid['percentage_of_pre_covid'].rolling(window=30).mean()

# Pre-compute the data for the plots
monthly_counts = df.groupby(['MONTH', 'COUNTY_DESC', 'REPORT_TYPE']).size().reset_index()
monthly_counts.columns = ['MONTH', 'COUNTY_DESC', 'REPORT_TYPE', 'COUNT']

# Pre-compute the data for the plots
daily_counts = df.groupby(['DAY', 'COUNTY_DESC', 'REPORT_TYPE']).size().reset_index()
daily_counts.columns = ['DAY', 'COUNTY_DESC', 'REPORT_TYPE', 'COUNT']

In [6]:
# In FeatureEngineering.ipynb
hourly_counts.to_csv('assets/hourly_counts.csv', index=False)
post_covid.to_csv('assets/post_covid.csv', index=False)
county_counts.to_csv('assets/county_counts.csv', index=False)
monthly_counts.to_csv('assets/monthly_counts.csv', index=False)
daily_counts.to_csv('assets/daily_counts.csv', index=False)